In [59]:
from azureml.core import Workspace, Dataset, Datastore
import pandas as pd
import logging


def get_data():
    subscription_id = 'd2706c67-acfc-4bd3-9067-3ff6ac190bc9'
    resource_group = 'capstone-project'
    workspace_name = 'capstone-project'

    workspace = Workspace(subscription_id, resource_group, workspace_name)

    datastore = Datastore.get(workspace, "workspaceworkingdirectory")
    dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'Users/hualcosa/nd00333-capstone/data/Walmart Data Analysis and Forcasting.csv'))
    df = dataset.to_pandas_dataframe() 
    return df

In [63]:
def process_data(df):
    '''
    This function formats the dataframe, adding past 8 weeks of sales as lagged features
    and 4 weeks of future sales as the label column.

    Return:
    X. Training data with features + lagged sales
    y. vector with the next 4 weeks of sales
    '''
    
    df_with_windows = []
    for store_num in df.Store.unique():
        store_df = df[df.Store == store_num].copy()
        # making lag features
        for i in range(1, 9):
            store_df[f'Weekly_Sales_t-{i}'] = store_df['Weekly_Sales'].shift(i)
        # making future_time_steps
        for i in range(1,4):
            store_df[f'Weekly_Sales_t+{i}'] = store_df['Weekly_Sales'].shift(-i)

        df_with_windows.append(store_df)

    df_with_windows = pd.concat(df_with_windows).dropna()
    # renaming first future value, to follow the same pattern as the other columns
    df_with_windows.rename(columns={"Weekly_Sales":"Weekly_Sales_t+0"}, inplace=True)
    df_with_windows = df_with_windows[['Store', 'Date', 'Holiday_Flag', 'Temperature',
                                        'Fuel_Price', 'CPI', 'Unemployment', 'Weekly_Sales_t-1',
                                        'Weekly_Sales_t-2', 'Weekly_Sales_t-3', 'Weekly_Sales_t-4',
                                        'Weekly_Sales_t-5', 'Weekly_Sales_t-6', 'Weekly_Sales_t-7',
                                        'Weekly_Sales_t-8', 'Weekly_Sales_t+0','Weekly_Sales_t+1', 'Weekly_Sales_t+2',
                                        'Weekly_Sales_t+3']]

    # future columns filter
    ftr = df_with_windows.columns.str.match(r'.+t\+\d')
    # making label vector
    y = df_with_windows.loc[:, ftr].apply(lambda row: list(row), axis=1)
    df_with_windows = df_with_windows.loc[:, ~ftr]
    # making training data
    X = df_with_windows.values

    return X, y


In [66]:
X

,Store,Date,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Weekly_Sales_t-1,Weekly_Sales_t-2,Weekly_Sales_t-3,Weekly_Sales_t-4,Weekly_Sales_t-5,Weekly_Sales_t-6,Weekly_Sales_t-7,Weekly_Sales_t-8
8,1,2010-04-02,0,62.27,2.719,210.820450,7.808,1404429.92,1472515.79,1439541.59,1554806.68,1409727.59,1611968.17,1641957.44,1643690.90
9,1,2010-04-09,0,65.86,2.770,210.622857,7.808,1594968.28,1404429.92,1472515.79,1439541.59,1554806.68,1409727.59,1611968.17,1641957.44
10,1,2010-04-16,0,66.32,2.808,210.488700,7.808,1545418.53,1594968.28,1404429.92,1472515.79,1439541.59,1554806.68,1409727.59,1611968.17
11,1,2010-04-23,0,64.84,2.795,210.439123,7.808,1466058.28,1545418.53,1594968.28,1404429.92,1472515.79,1439541.59,1554806.68,1409727.59
12,1,2010-04-30,0,67.41,2.780,210.389546,7.808,1391256.12,1466058.28,1545418.53,1594968.28,1404429.92,1472515.79,1439541.59,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6427,45,2012-09-07,1,75.70,3.911,191.577676,8.684,734297.87,718232.26,722496.93,733037.32,725729.51,711671.58,737613.65,749817.08
6428,45,2012-09-14,0,67.87,3.948,191.699850,8.684,766512.66,734297.87,718232.26,722496.93,733037.32,725729.51,711671.58,737613.65
6429,45,2012-09-21,0,65.32,4.038,191.856704,8.684,702238.27,766512.66,734297.87,718232.26,722496.93,733037.32,725729.51,711671.58
6430,45,2012-09-28,0,64.88,3.997,192.013558,8.684,723086.20,702238.27,766512.66,734297.87,718232.26,722496.93,733037.32,725729.51


In [50]:
y

8       [1594968.28, 1545418.53, 1466058.28, 1391256.12]
9       [1545418.53, 1466058.28, 1391256.12, 1425100.71]
10      [1466058.28, 1391256.12, 1425100.71, 1603955.12]
11       [1391256.12, 1425100.71, 1603955.12, 1494251.5]
12       [1425100.71, 1603955.12, 1494251.5, 1399662.07]
                              ...                       
6427         [766512.66, 702238.27, 723086.2, 713173.95]
6428         [702238.27, 723086.2, 713173.95, 733455.07]
6429         [723086.2, 713173.95, 733455.07, 734464.36]
6430        [713173.95, 733455.07, 734464.36, 718125.53]
6431        [733455.07, 734464.36, 718125.53, 760281.43]
Length: 5940, dtype: object

In [64]:
def main():
    logging.basicConfig(
        level=logging.INFO,
        format="%(name)s - %(asctime)s - %(levelname)s - %(message)s",
        datefmt="%d-%b-%y %H:%M:%S",
    )

    logging.info('Getting data...')
    df = get_data()
    logging.info('formatting data...')
    X, y = process_data(df)
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    # logging.info('training model...')

In [65]:
if __name__ == '__main__':
    main()

X shape: (5940, 15)
y shape: (5940,)


In [58]:
main()

In [ ]:
# Tr